<a href="https://colab.research.google.com/github/jmarrietar/ocular/blob/master/notebooks/PAWS_CIFAR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Drive
from google.colab import drive
from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

In [2]:
drive.mount('/content/drive')
auth.authenticate_user()

Mounted at /content/drive


In [3]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.11 MiB | 21.40 MiB/s, done.
Resolving deltas: 100% (5464/5464), done.
/content/apex
Created temporary directory: /tmp/pip-ephem-wheel-cache-adaevmkl
Created temporary directory: /tmp/pip-req-tracker-1zce5h17
Created requirements tracker '/tmp/pip-req-tracker-1zce5h17'
Created temporary directory: /tmp/pip-install-ps7y7jom
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-6vi4rlud
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-1zce5h17'
    Running setup.py (path:/tmp/pip-req-build-6vi4rlud/setup.py) egg_info for package from file:///content/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.8.1+cu101


    running egg_info
    creating /tmp/pip-re

In [4]:
pip install PyYAML

In [5]:
cd ..

/content


In [6]:
!git clone -b develop https://github.com/jmarrietar/suncet.git

Cloning into 'suncet'...
remote: Enumerating objects: 136, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 136 (delta 67), reused 107 (delta 46), pack-reused 0
Receiving objects: 100% (136/136), 1.08 MiB | 5.37 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [7]:
cd suncet

/content/suncet


In [8]:
pwd

'/content/suncet'

In [9]:
!mkdir datasets
!mkdir datasets/cifar10-data

In [10]:
!wget --no-check-certificate \
    https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz \
    -O /content/suncet/datasets/cifar10-data/cifar-10-python.tar.gz

--2021-06-01 04:36:24--  https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
Resolving www.cs.toronto.edu (www.cs.toronto.edu)... 128.100.3.30
Connecting to www.cs.toronto.edu (www.cs.toronto.edu)|128.100.3.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 170498071 (163M) [application/x-gzip]
Saving to: ‘/content/suncet/datasets/cifar10-data/cifar-10-python.tar.gz’

/content/suncet/dat 100%[===================>] 162.60M  17.3MB/s    in 11s     

2021-06-01 04:36:36 (15.1 MB/s) - ‘/content/suncet/datasets/cifar10-data/cifar-10-python.tar.gz’ saved [170498071/170498071]



In [11]:
 cd /content/suncet/datasets/cifar10-data/

/content/suncet/datasets/cifar10-data


In [12]:
!tar xvf cifar-10-python.tar.gz

cifar-10-batches-py/
cifar-10-batches-py/data_batch_4
cifar-10-batches-py/readme.html
cifar-10-batches-py/test_batch
cifar-10-batches-py/data_batch_3
cifar-10-batches-py/batches.meta
cifar-10-batches-py/data_batch_2
cifar-10-batches-py/data_batch_5
cifar-10-batches-py/data_batch_1


In [13]:
cd ../../

/content/suncet


In [14]:
pip install -U PyYAML

     |████████████████████████████████| 645kB 2.9MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [15]:
!mkdir logs

In [16]:
import argparse

import torch.multiprocessing as mp

import pprint
import yaml

from src.paws_train import main as paws
from src.suncet_train import main as suncet
from src.fine_tune import main as fine_tune
from src.snn_fine_tune import main as snn_fine_tune

from src.utils import init_distributed

In [17]:
parser = argparse.ArgumentParser()
parser.add_argument(
    '--fname', type=str,
    help='name of config file to load',
    default='configs.yaml')
parser.add_argument(
    '--devices', type=str, nargs='+', default=['cuda:0'],
    help='which devices to use on local machine')
parser.add_argument(
    '--sel', type=str,
    help='which script to run',
    choices=[
        'paws_train',
        'suncet_train',
        'fine_tune',
        'snn_fine_tune'
    ])

_StoreAction(option_strings=['--sel'], dest='sel', nargs=None, const=None, default=None, type=<class 'str'>, choices=['paws_train', 'suncet_train', 'fine_tune', 'snn_fine_tune'], help='which script to run', metavar=None)

In [18]:
args = parser.parse_args(['--sel', 'paws_train',
                            '--fname', 'configs/paws/cifar10_train.yaml'
])

In [19]:
def process_main(rank, sel, fname, world_size, devices):
    import os
    #os.environ['CUDA_VISIBLE_DEVICES'] = str(devices[rank].split(':')[-1])
    os.environ['CUDA_VISIBLE_DEVICES'] = str(args.devices[rank].split(':')[-1]) # YO

    import logging
    logging.basicConfig()
    logger = logging.getLogger()

    logger.info(f'called-params {sel} {fname}')

    # -- load script params
    params = None
    with open(fname, 'r') as y_file:
        params = yaml.load(y_file, Loader=yaml.FullLoader)
        logger.info('loaded params...')
        if rank == 0:
            pp = pprint.PrettyPrinter(indent=4)
            pp.pprint(params)

    print("loggin folder {}".format(params['logging']['folder']))

    if rank == 0:
        dump = os.path.join(params['logging']['folder'], f'params-{sel}.yaml')
        with open(dump, 'w') as f:
            yaml.dump(params, f)

    world_size, rank = init_distributed(rank_and_world_size=(rank, world_size))

    # -- make sure all processes correctly initialized torch-distributed
    logger.info(f'Running {sel} (rank: {rank}/{world_size})')

    # -- turn off info-logging for ranks > 0, otherwise too much std output
    if rank == 0:
        logger.setLevel(logging.INFO)
    else:
        logger.setLevel(logging.ERROR)

    if sel == 'paws_train':
        return paws(params)
    elif sel == 'suncet_train':
        return suncet(params)
    elif sel == 'fine_tune':
        return fine_tune(params)
    elif sel == 'snn_fine_tune':
        return snn_fine_tune(params)

In [20]:
num_gpus = len(args.devices)

In [21]:
process_main(0, args.sel, args.fname, num_gpus, args.devices)

INFO:root:called-params paws_train configs/paws/cifar10_train.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 10,
                     'me_max': False,
                     'sharpen': 0.25,
                     'supervised_imgs_per_class': 64,
                     'supervised_views': 2,
                     'temperature': 0.1,
                     'unsupervised_batch_size': 256},
    'data': {   'color_jitter_strength': 0.5,
                'data_seed': 152,
                'dataset': 'cifar10',
                'image_folder': 'cifar10-data/',
                'label_smoothing': 0.1,
                'multicrop': 6,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'cifar10_subsets/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.92},
    'logging': {   'folder': '/content/drive/MyDrive/Colab '
                             'Notebooks/PAWS/logs/',
                   

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


INFO:root:Using cifar10_subsets/spc.4000_split.152.txt
INFO:root:num-labeled 4000
INFO:root:num-labeled target 0 400
INFO:root:num-labeled target 1 400
INFO:root:num-labeled target 2 400
INFO:root:num-labeled target 3 400
INFO:root:num-labeled target 4 400
INFO:root:num-labeled target 5 400
INFO:root:num-labeled target 6 400
INFO:root:num-labeled target 7 400
INFO:root:num-labeled target 8 400
INFO:root:num-labeled target 9 400
INFO:root:min. labeled indices 400
INFO:root:iterations per epoch: 195
INFO:root:Epoch 1
INFO:root:len.supervised_loader: 195
INFO:root:[1,     0] loss: 2.260 (2.260 0.000) (4649 ms; 13500 ms)
INFO:root:[1,     0] lr_stats: 0.038 (1.18e-02, 1.25e-01)
INFO:root:[1,    10] loss: 2.181 (2.181 0.000) (1251 ms; 1304 ms)
INFO:root:[1,    10] lr_stats: 0.031 (1.07e-02, 9.07e-02)
INFO:root:[1,    20] loss: 2.134 (2.134 0.000) (1103 ms; 901 ms)
INFO:root:[1,    20] lr_stats: 0.027 (9.63e-03, 5.96e-02)
INFO:root:[1,    30] loss: 2.065 (2.065 0.000) (1048 ms; 632 ms)
INFO:

In [22]:
args = parser.parse_args(['--sel', 'snn_fine_tune',
                          '--fname', 'configs/paws/cifar10_snn.yaml'
])

In [23]:
args

Namespace(devices=['cuda:0'], fname='configs/paws/cifar10_snn.yaml', sel='snn_fine_tune')

In [24]:
process_main(0, args.sel, args.fname, num_gpus, args.devices)

INFO:root:called-params snn_fine_tune configs/paws/cifar10_snn.yaml
INFO:root:loaded params...
{   'criterion': {   'classes_per_batch': 10,
                     'supervised_batch_size': 64,
                     'supervised_views': 2,
                     'temperature': 0.1},
    'data': {   'data_seed': 152,
                'dataset': 'cifar10_fine_tune',
                'image_folder': 'cifar10-data/',
                'label_smoothing': 0.0,
                'normalize': True,
                'root_path': 'datasets/',
                'subset_path': 'cifar10_subsets/',
                'unique_classes_per_rank': False,
                'unlabeled_frac': 0.92},
    'logging': {   'folder': '/content/drive/MyDrive/Colab '
                             'Notebooks/PAWS/logs/',
                   'pretrain_path': 'paws-latest.pth.tar',
                   'write_tag': 'paws-latest-SNN'},
    'meta': {   'copy_data': True,
                'device': 'cuda:0',
                'load_checkpoint': Fa

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


INFO:root:[1] (val: 94.640%)
INFO:root:[1,     0] (loss: 0.008)
INFO:root:[2] (val: 94.830%)
INFO:root:[2,     0] (loss: 0.005)
INFO:root:[3] (val: 94.700%)
INFO:root:[3,     0] (loss: 0.002)
INFO:root:[4] (val: 94.860%)
INFO:root:[4,     0] (loss: 0.003)
INFO:root:[5] (val: 94.750%)
INFO:root:[5,     0] (loss: 0.002)
INFO:root:[6] (val: 94.520%)
INFO:root:[6,     0] (loss: 0.001)
INFO:root:[7] (val: 94.700%)
INFO:root:[7,     0] (loss: 0.003)
INFO:root:[8] (val: 94.810%)
INFO:root:[8,     0] (loss: 0.001)
INFO:root:[9] (val: 94.760%)
INFO:root:[9,     0] (loss: 0.001)
INFO:root:[10] (val: 94.700%)
INFO:root:[10,     0] (loss: 0.001)
INFO:root:[11] (val: 94.850%)
INFO:root:[11,     0] (loss: 0.001)
INFO:root:[12] (val: 94.690%)
INFO:root:[12,     0] (loss: 0.001)
INFO:root:[13] (val: 94.760%)
INFO:root:[13,     0] (loss: 0.001)
INFO:root:[14] (val: 94.830%)
INFO:root:[14,     0] (loss: 0.001)
INFO:root:[15] (val: 94.700%)
INFO:root:[15,     0] (loss: 0.001)
INFO:root:[16] (val: 94.670%